<center>
    <h2> Aprendizado de Máquina </h2>
    <hr >
    <h1>Task 05 - Viral Tweets Prediction Challenge</h1>
    <hr >
    <h4>Diego J Talarico Ferreira- 3166561</h4>
    <h4>Gabriel Castro Gulin Rosa - 11218181</h4>
    <h4>Júlio Trevisan Centanin - 11218240</h4>
    <h4>Matheus Victal Cerqueira - 10276661</h4>
    <h4>Murilo Henrique Soave - 10688813</h4>
    <hr >
    <img src= 'https://img.olhardigital.com.br/wp-content/uploads/2021/05/shutterstock_1963706737-1000x450.jpg' width="700" >
    <hr >
</center>

# Predição de Tweets Virais

___


## Análise Descritiva

Uma análise exploratória descritiva bem feita é extremamente importante na identificação de tendências e comportamentos dos dados de interesse, além de localizar possíveis problemas como valores nulos, os quais podem ser tratados em uma etapa de pré-processamento. Assim, podemos ter uma visão holística dos dados, o que permite que o ajuste de modelos de aprendizado de máquina seja mais simples. Como aqui temos uma base de dados considerávelmente grande, devemos analisar cuidadosamente a estrutura e comportamento das observações contidas em tal conjunto, a fim de facilitar a modelagem posteriormente.

In [ ]:
# Bibliotecas básicas e para análise descritiva
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Análise de correlações e redução de dimensionalidade
from sklearn.decomposition import PCA

# Balanceamento de dados
from imblearn.over_sampling import SMOTE

# Modelagem

from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

### Uma breve observação da estrutura dos dados

In [ ]:
# Caminho base
t_path = '../input/viral-tweets-prediction-dataset/Dataset/Tweets/'
u_path = '../input/viral-tweets-prediction-dataset/Dataset/Users/'

# Dados de treino
train_tweets = pd.read_csv(t_path + 'train_tweets.csv')
train_tweets_vectorized_media = pd.read_csv(t_path + 'train_tweets_vectorized_media.csv')
train_tweets_vectorized_text = pd.read_csv(t_path + 'train_tweets_vectorized_text.csv')

# Dados de usuário
users = pd.read_csv(u_path + 'users.csv')
user_vectorized_descriptions = pd.read_csv(u_path + 'user_vectorized_descriptions.csv')
user_vectorized_profile_images = pd.read_csv(u_path + 'user_vectorized_profile_images.csv')

Os dados que aqui estamos estudando estão distribuídos em seis conjuntos que estão interrelacionados por dois atributos de identificação: o identificador de usuário (user_id) e o identificador de tweet (tweet_id). Tais conjuntos podem ser agrupados da seguinte forma:

___ 
Tweets:                                   

 * test_tweets_vectorized_media.csv         
 * test_tweets_vectorized_text.csv          
 * test_tweets.csv                          
 * train_tweets_vectorized_media.csv 
 * train_tweets_vectorized_text.csv
 * train_tweets.csv
 
Users:

* user_vectorized_descriptions.csv
* user_vectorized_profile_images.csv
* users.csv
___

Vejamos como esses conjuntos de dados estão estruturados a partir de uma amostra de cada um deles (para os conjuntos que possuam um par _train-test_, iremos apenas observar o componente referente ao treino por agora).

In [ ]:
train_tweets_vectorized_media.sample(3)

In [ ]:
train_tweets_vectorized_text.sample(3)

In [ ]:
train_tweets.sample(3)

In [ ]:
users.sample(3)

In [ ]:
user_vectorized_descriptions.sample(3)

In [ ]:
user_vectorized_profile_images.sample(3)

Os conjuntos de dados referentes à informações vetorizadas oferecem poucas conclusões passíveis de interpretação qualitativa, assim sendo, iremos focar nossa atenção, momentanemente, aos conjuntos _users.csv_ e _train_tweets.csv_, os quais possuem features interpretáveis e que podem gerar conclusões interessnates antes de ajustarmos algum modelo propriamente dito.

### Investigação qualitativa e quantitativa dos dados dos conjuntos de dados não vetorizados

Os conjuntos _users.csv_ e _train_tweets.csv_, como já comentado, possuem atributos que permitem uma interpretação qualitativa em relação ao comportamento dos dados. Primeiramente, vejamos algumas características básicas:

In [ ]:
train_tweets.info()

In [ ]:
train_tweets.shape

É notório que a única coluna que apresenta dados faltantes é a coluna _tweet_topic_ids_. Observando-se nas amostras coletadas anteriormente deste conjunto, podemos perceber que as observações contidas nessa coluna possuem uma estrutura de lista de identificadores. Assim sendo, seu tratamento será mais complicado e deve ser investigado se sua utilização é realmente necessária. 

Agora, estudemos o comportamento desses _features_ a partir de alguns gráficos. Primeiramente, iremos analisar como as proporções entre as categorias de viralidade (de 1 a 5) se comportam de acordo com as covariáveis temporais dos dados.

In [ ]:
plt.style.use('ggplot')

In [ ]:
plt.figure(figsize=(12, 10))
sns.countplot(x = 'virality', data = train_tweets, palette="viridis")

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(18, 8))

sns.histplot(x="tweet_created_at_year", hue = 'virality', multiple = 'stack', discrete = True,
             ax = axs[0,0], data=train_tweets, palette = "viridis")
sns.histplot(x="tweet_created_at_month", hue = 'virality', multiple = 'stack', discrete = True,
             ax = axs[0,1], legend = False, data=train_tweets, palette = "viridis")
sns.histplot(x="tweet_created_at_day", hue = 'virality', multiple = 'stack', discrete = True,
             ax = axs[1,0], legend = False, data=train_tweets, palette = "viridis")
sns.histplot(x="tweet_created_at_hour", hue = 'virality', multiple = 'stack', discrete = True,
             ax = axs[1,1], legend = False, data=train_tweets, palette = "viridis")
plt.show()

Observando-se os histogramas acima, os quais estão divididos pelo nível de viralidade, é possível notar que, por mais que o número de observações seja diferente para cada ano e para cada hora, as proporções entre as categorias de viralidade não parecem variar de forma sifnificativa a ponto de podermos atrelar tais atributos diretamente ao grau de viralidade. Um fenômeno semelhante ocorre para o caso das covariáveis de dia e mês, porém, nesse caso o comportamento parece se repetir para quase todo o espaço amostral, principalemente para o caso do dia do mês. Como tal variabilidade é pequena, é possível que tais covariáveis não estejam fortemente relacionadas com o grau de viralidade, pelo menos, não de uma forma trivial.

Agora, vamos investigar alguns tipos de correlações entre variáveis. A ideia é tentar identificar algum padrão entre os níveis de viralidade e os atributos do conjunto de dados. Assim, investiguemos a correlação de [pearson](https://pt.wikipedia.org/wiki/Coeficiente_de_correla%C3%A7%C3%A3o_de_Pearson), a correlação de [spearman](https://pt.wikipedia.org/wiki/Coeficiente_de_correla%C3%A7%C3%A3o_de_postos_de_Spearman) e a correlação de [kendall](https://pt.wikipedia.org/wiki/Coeficiente_de_correla%C3%A7%C3%A3o_tau_de_Kendall) entre a variável de viralidade e os demais fatores.

In [ ]:
single1 = train_tweets.corr(method = "pearson")[['virality']].sort_values(by='virality', ascending=False)
single2 = train_tweets.corr(method = "spearman")[['virality']].sort_values(by='virality', ascending=False)
single3 = train_tweets.corr(method = "kendall")[['virality']].sort_values(by='virality', ascending=False)

In [ ]:
plt.figure(figsize=(15, 12))
fig, axs = plt.subplots(1, 3, figsize=(18, 8))

sns.heatmap(single1, vmin=-1, vmax=1, annot=True, ax = axs[0], cbar = False)
sns.heatmap(single2, vmin=-1, vmax=1, annot=True, ax = axs[1], cbar = False, yticklabels=False)
sns.heatmap(single3, vmin=-1, vmax=1, annot=True, ax = axs[2], yticklabels=False)

axs[0].set_title("Pearson")
axs[1].set_title("Spearman")
axs[2].set_title("Kendall")

plt.show(True)

Observando-se os mapas de correlação acima, podemos perceber que, para nenhum dos coeficientes de correlação acima, há retultados que indiquem forte relação entre a viralidade e os demais _features_. Assim sendo, se existir uma relação, ela provavelmente não é trivial e um modelo mais robusto talvez seja necessário para identificar como estas variáveis se relacionam.

Analisemos agora as colunas _tweet_has_attachment_ e _tweet_attachment_class_ em mais detalhe

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

p1 = sns.histplot(x="tweet_has_attachment", hue = 'virality', multiple = 'stack', discrete = True,
                  ax = axs[0], data=train_tweets, palette = 'viridis')
p1.set(xticks = [0,1])

sns.histplot(x="tweet_attachment_class", hue = 'virality', multiple = 'stack', discrete = True,
             ax = axs[1], legend = False, data=train_tweets, palette = 'viridis')

plt.show(True)

Como esperado, não há um padrão trivial significativo que permita visualizarmos uma relação direta entre a viralidade e tais colunas. Além disso, quando uma linha tem a coluna 'tweet_has_attachment' False, temos que a coluna 'tweet_attachment_class' é C para essa linha. Logo, essas duas colunas tem uma alta correlação. Quase não há linhas com 'tweet_attachment_class' com valor 'B'. 

Por fim, utilizemos de PCA para verificar se a correlação entre as features é alta e se é possível reduzir a dimensionalidade do conjunto com pouca perda de informação.

In [ ]:
train_tweets.head(3)

In [ ]:
np.matrix(train_tweets.drop(['tweet_attachment_class', 'tweet_has_attachment'], axis = 1).iloc[:,0:10]) # aqui pegamos todas 
# as colunas menos a de viralidade e os tweet_topic_ids para aplicar o PCA. Além disso removem-se tweet_attachment_class e 
# tweet_has_attachment os quais não são numéricos.

In [ ]:
from sklearn.decomposition import PCA
PCA_copy = train_tweets.copy()

pca = PCA(n_components=2)

X = np.matrix(PCA_copy.drop(['tweet_attachment_class', 'tweet_has_attachment'], axis = 1).iloc[:,0:10]) 
pca.fit(X)

print(np.round(pca.explained_variance_ratio_,6))

PCA1 = pca.transform(X)[:,0]
PCA2 = pca.transform(X)[:,1]
PCA_copy["Component 1"] = PCA1
PCA_copy["Component 2"] = PCA2

In [ ]:
PCA_copy.head(3)

Considerando-se as observações de acordo com a viralidade observada.

In [ ]:
plt.figure(figsize=(15, 7))

fig, axs = plt.subplots(1, 5, figsize=(18, 8))
sns.scatterplot(data=PCA_copy[PCA_copy['virality']==1], x="Component 1", y="Component 2", palette = 'viridis', ax = axs[0])
sns.scatterplot(data=PCA_copy[PCA_copy['virality']==2], x="Component 1", y="Component 2", palette = 'viridis', ax = axs[1])
sns.scatterplot(data=PCA_copy[PCA_copy['virality']==3], x="Component 1", y="Component 2", palette = 'viridis', ax = axs[2])
sns.scatterplot(data=PCA_copy[PCA_copy['virality']==4], x="Component 1", y="Component 2", palette = 'viridis', ax = axs[3])
sns.scatterplot(data=PCA_copy[PCA_copy['virality']==5], x="Component 1", y="Component 2", palette = 'viridis', ax = axs[4])

axs[0].set_title("Virality = 1")
axs[1].set_title("Virality = 2")
axs[2].set_title("Virality = 3")
axs[3].set_title("Virality = 4")
axs[4].set_title("Virality = 5")

Considerando-se todos os tipos de viralidade.

In [ ]:
plt.figure(figsize=(15, 7))
sns.scatterplot(data=PCA_copy, x="Component 1", y="Component 2", palette = 'viridis', hue = "virality")

Aplicando-se PCA, pode-se perceber pela alta variância explicada pelas duas componentes principais mais explicativas que existe uma correlação considerável entre os features que passaram pelo processo. Porém, não é possível identificar um padrão de viralidade trivial utilizando-se apenas das duas componentes principais, como mostram on gráficos de dispersão acima.

<br>
<br>

Agora, investiguemos o conjunto de dados _users_.

In [ ]:
users.info()

In [ ]:
users.shape

Aqui não há a ocorrência de dados faltantes, o que irá facilitar, de certa forma, o pré-processamento. Façamos alguns histogramas para as colunas relacionadas à relevância do usuário na rede (_user_like_count_, _user_followers_count_, _user_following_count_, _user_listed_on_count_).

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(15, 8))

sns.histplot(users, x = 'user_like_count', ax = axs[0,0], color = 'darkviolet')
sns.histplot(users, x = 'user_followers_count', ax = axs[0,1], color = 'darkviolet')
sns.histplot(users, x = 'user_following_count', ax = axs[1,0], color = 'darkviolet')
sns.histplot(users, x = 'user_listed_on_count', ax = axs[1,1], color = 'darkviolet')

É interessante perceber que tais features obedecem distribuições livres de escala, o que é comum em sistemas estruturados em rede tais como redes sociais. Isso quer dizer que muitos usuários possuem indicadores baixos de conectividade enquanto uma gama baixa de poucos usuários posssuem indicadores de conectividade altos, configurando-se como _hubs_ da rede. Tais _hubs_ estão relacionados à todo tipo de processo dinâmico que ocorre na rede e sua investigação pode levar à uma melhor compreensão da topologia do grafo em questão. É esperado que exista uma correlação considerável entre os features aqui tratados. 

Em análise de redes sociais, tais _hubs_ são extremamente importantes, pois podem corresponder à super-propagadores de opinião, influenciadores de tendências e coesão de comunidades. Porém, é preciso mais do que conhecer os hubs de uma rede para prever a viralidade de um tweet, já que o sistema envolvido é extremamente complexo e os processos dinâmicos nele envolvidos são não triviais.

Muitos estudos estão voltados para a análise de redes sociais por meio de estruturas de redes complexas. Entre eles, temos o [estudo](https://osome.iu.edu/demos/echo/) realizado pelo observatório em mídias sociais [OSoMe](https://osome.iu.edu/tools) da Universidade de Indiana, onde é feita uma simulação de câmaras de eco, as quais são extremamente interessantes na área de comunicação e sociologia, sendo muitas vezes formadores de opinião pública e responsáveis pela emergência de comportamentos coletivos diversos. 

In [ ]:
from sklearn.decomposition import PCA

PCA_copy = users.copy()

pca = PCA(n_components=2)

X = np.matrix(PCA_copy.iloc[:,1:11]) # análise das colunas, menos a coluna de identificação
pca.fit(X)

print(np.round(pca.explained_variance_ratio_,6))

In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(users.corr() , annot = True)
plt.show()

Como esperado, existe correlação moderada entre as colunas do conjunto de dados em questão, o que leva aos valores de variâcia explicada pelas duas componentes principais mais explicativas do PCA.

### Investigação quantitativa dos dados dos conjuntos de dados vetorizados

Como já discutido, uma análise qualitativa dos conjuntos de dados vetorizados não é algo que iremos fazer devido ao  significado não muito intuitivo do que as colunas representam. Assim sendo, iremos apenas investigar a correlação entre os _features_ de cada conjunto.

### _train_tweets_vectorized_media_

In [ ]:
train_tweets_vectorized_media.info()

In [ ]:
train_tweets_vectorized_media.shape

In [ ]:
from sklearn.decomposition import PCA

PCA_copy = train_tweets_vectorized_media.copy()

pca = PCA(n_components=2)

X = np.matrix(PCA_copy.iloc[:,0:2050]) # análise das colunas, menos a coluna de identificação
pca.fit(X)

print(np.round(pca.explained_variance_ratio_,6))

In [ ]:
sns.heatmap(train_tweets_vectorized_media.corr() , annot = False)
plt.title("Mapa de calor de correlação de Pearson (train_tweets_vectorized_media)")
plt.show()

### _train_tweets_vectorized_text_

In [ ]:
train_tweets_vectorized_text.info()

In [ ]:
train_tweets_vectorized_text.shape

In [ ]:
from sklearn.decomposition import PCA

PCA_copy = train_tweets_vectorized_text.copy()

pca = PCA(n_components=2)

X = np.matrix(PCA_copy.iloc[:,0:769]) # análise das colunas, menos a coluna de identificação
pca.fit(X)

print(np.round(pca.explained_variance_ratio_,6))

In [ ]:
sns.heatmap(train_tweets_vectorized_text.corr() , annot = False)
plt.title("Mapa de calor de correlação de Pearson (train_tweets_vectorized_text)")
plt.show()

### _user_vectorized_descriptions_

In [ ]:
user_vectorized_descriptions.info()

In [ ]:
user_vectorized_descriptions.shape

In [ ]:
from sklearn.decomposition import PCA

PCA_copy = user_vectorized_descriptions.copy()

pca = PCA(n_components=2)

X = np.matrix(PCA_copy.iloc[:,0:769]) # análise das colunas, menos a coluna de identificação
pca.fit(X)

print(np.round(pca.explained_variance_ratio_,6))

In [ ]:
sns.heatmap(user_vectorized_descriptions.corr() , annot = False)
plt.title("Mapa de calor de correlação de Pearson (user_vectorized_descriptions)")
plt.show()

### _user_vectorized_profile_images_

In [ ]:
user_vectorized_profile_images.info()

In [ ]:
user_vectorized_profile_images.shape

In [ ]:
from sklearn.decomposition import PCA

PCA_copy = user_vectorized_profile_images.copy()

pca = PCA(n_components=2)

X = np.matrix(PCA_copy.iloc[:,0:2049]) # análise das colunas, menos a coluna de identificação
pca.fit(X)

print(np.round(pca.explained_variance_ratio_,6))

In [ ]:
sns.heatmap(user_vectorized_profile_images.corr(), annot = False)
plt.title("Mapa de calor de correlação de Pearson (user_vectorized_profile_images)")
plt.show()

As duas componentes principais mais explicativas para os quatro casos explicam uma enorme parte da variabilidade (mais de 85% para todos os casos, com três deles acima de 99%) de cada conjunto, o que indica que há correlações consideráveis entre os features. Tal conclusão é reafirmada pelos mapas de calor de correlação de Pearson.

### A feature "influência"

In [ ]:
users["influencia"] = users["user_followers_count"]/users["user_following_count"]
users["log10_influencia"] = np.log10(users["user_followers_count"]/users["user_following_count"])

In [ ]:
users.replace([np.inf, -np.inf], np.nan, inplace=True)
users.dropna()

In [ ]:
plt.figure(figsize=(15, 8))
sns.histplot(users, x = 'log10_influencia', color = "darkviolet")

## Primeira solução

A ideia é utilizar todas as informações disponíveis. Vamos relizar um PCA e juntar todos os dataframes em um só.

### Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import imblearn
import warnings
import time
warnings.filterwarnings('ignore')

### Lendo os dados

Vamos tratar cada banco de dados individualmente e depois juntá-los

In [ ]:
train_tweets = pd.read_csv('../input/viral-tweets-prediction-dataset/Dataset/Tweets/train_tweets.csv')
train_tweets_vectorized_media = pd.read_csv('../input/viral-tweets-prediction-dataset/Dataset/Tweets/train_tweets_vectorized_media.csv')
train_tweets_vectorized_text = pd.read_csv('../input/viral-tweets-prediction-dataset/Dataset/Tweets/train_tweets_vectorized_text.csv')
users = pd.read_csv('../input/viral-tweets-prediction-dataset/Dataset/Users/users.csv')
user_vectorized_profile_images = pd.read_csv('../input/viral-tweets-prediction-dataset/Dataset/Users/user_vectorized_profile_images.csv')
user_vectorized_descriptions = pd.read_csv('../input/viral-tweets-prediction-dataset/Dataset/Users/user_vectorized_descriptions.csv')

### Pré-processamento de dados

Vamos primeiro utilizar o PCA para eliminar qualquer correlação entre as colunas dos dados que estão vetorizados. Depois, vamos juntar todos os dataframes em um só, para montar o modelo.

**Arquivo tweets_vectorized_media.csv**

In [ ]:
from sklearn.preprocessing import StandardScaler
features = [coluna for coluna in train_tweets_vectorized_media.columns if coluna not in ['media_id', 'tweet_id']]

#Selecionando as colunas
X_vectorized_media = train_tweets_vectorized_media[features].to_numpy()

#Padronizando para o PCA
X_vectorized_media = StandardScaler().fit_transform(X_vectorized_media)

In [ ]:
from sklearn.decomposition import PCA
#Fazendo o PCA
pca = PCA(0.95)
components = pca.fit_transform(X_vectorized_media)

In [ ]:
#Excluindo as features antigas, para adicionar as componentes principais
train_tweets_vectorized_media = train_tweets_vectorized_media.drop(features, axis = 1)

In [ ]:
#Adicionando as componentes principais ao dataframe 'train_tweets_vectorized_media'
columns_components = ['img_pca_%i' % i for i in range(len(pca.explained_variance_ratio_))]
df_components = pd.DataFrame(data = components, columns = columns_components)

In [ ]:
#Juntando os dataframes train_tweets_vectorized_media com os valores das componentes do pca
train_tweets_vectorized_media = pd.merge(train_tweets_vectorized_media, df_components, left_index = True, right_index = True)

In [ ]:
train_tweets_vectorized_media.head(2)

Há tweets, com mais de uma mídia, vamos arrumar isso.

In [ ]:
#Numero de linhas com tweet_id unico
print('Tweets com uma midia:', train_tweets_vectorized_media['tweet_id'].nunique())
print('Tweets com mais de uma midia:', train_tweets_vectorized_media['tweet_id'].duplicated().sum())

In [ ]:
def fix_duplicates(df):
    ''' Funcao que substitui tweets com mais de uma imagem e pega a media delas, de cada feature '''
    df_new = df.groupby('tweet_id').mean()
    return df_new

medias_without_duplicates = fix_duplicates(train_tweets_vectorized_media)

In [ ]:
print('Tweets com mais de uma midia:', medias_without_duplicates.index.duplicated().sum())

In [ ]:
def fix_missing_rows(df,df_missing):
    ''' Funcao que adiciona os tweets sem midia, com todas as features = 0, no dataset das midias '''
    missing = []
    for i in df['tweet_id']:
        if i not in df_missing.index:
            missing.append(i)
    
    #Dataframe com os tweets_id sem midias, com NaNs
    df_null = pd.DataFrame(index=missing)
    
    df_missing = pd.concat([df_missing, df_null])
    df_missing = df_missing.fillna(0)
    
    return df_missing

medias_without_duplicates = fix_missing_rows(train_tweets, medias_without_duplicates)

**Arquivo tweets_vectorized_text.csv**

In [ ]:
features = [coluna for coluna in train_tweets_vectorized_text.columns if coluna not in ['tweet_id']]

#Selecionando as colunas
X_vectorized_text = train_tweets_vectorized_text[features].to_numpy()

#Padronizando para o PCA
X_vectorized_text = StandardScaler().fit_transform(X_vectorized_text)

In [ ]:
#Fazendo o PCA
pca = PCA(0.95)
components = pca.fit_transform(X_vectorized_text)

In [ ]:
#Excluindo as features antigas, para adicionar as componentes principais
train_tweets_vectorized_text = train_tweets_vectorized_text.drop(features, axis = 1)

In [ ]:
#Adicionando as componentes principais ao dataframe 'train_tweets_vectorized_text'
columns_components = ['txt_pca_%i' % i for i in range(len(pca.explained_variance_ratio_))]
df_components = pd.DataFrame(data = components, columns = columns_components)

In [ ]:
#Juntando os dataframes train_tweets_vectorized_text com os valores das componentes do pca
train_tweets_vectorized_text = pd.merge(train_tweets_vectorized_text, df_components, left_index = True, right_index = True)

In [ ]:
train_tweets_vectorized_text.head(2)

**Arquivo user_vectorized_profile_images.csv**

In [ ]:
features = [coluna for coluna in user_vectorized_profile_images.columns if coluna not in ['user_id']]

#Selecionando as colunas
X_vectorized_user_img = user_vectorized_profile_images[features].to_numpy()

#Padronizando para o PCA
X_vectorized_user_img = StandardScaler().fit_transform(X_vectorized_user_img)

In [ ]:
#Fazendo o PCA
pca = PCA(0.95)
components = pca.fit_transform(X_vectorized_user_img)

In [ ]:
#Excluindo as features antigas, para adicionar as componentes principais
user_vectorized_profile_images = user_vectorized_profile_images.drop(features, axis = 1)

In [ ]:
#Adicionando as componentes principais ao dataframe 'user_vectorized_profile_images'
columns_components = ['user_img_pca_%i' % i for i in range(len(pca.explained_variance_ratio_))]
df_components = pd.DataFrame(data = components, columns = columns_components)

In [ ]:
#Juntando os dataframes user_vectorized_profile_images com os valores das componentes do pca
user_vectorized_profile_images = pd.merge(user_vectorized_profile_images, df_components, left_index = True, right_index = True)

**Arquivo user_vectorized_descriptions.csv**

In [ ]:
features = [coluna for coluna in user_vectorized_descriptions.columns if coluna not in ['user_id']]

#Selecionando as colunas
X_vectorized_user_desc = user_vectorized_descriptions[features].to_numpy()

#Padronizando para o PCA
X_vectorized_user_desc = StandardScaler().fit_transform(X_vectorized_user_desc)

In [ ]:
#Fazendo o PCA
pca = PCA(0.95)
components = pca.fit_transform(X_vectorized_user_desc)

In [ ]:
#Excluindo as features antigas, para adicionar as componentes principais
user_vectorized_descriptions = user_vectorized_descriptions.drop(features, axis = 1)

In [ ]:
#Adicionando as componentes principais ao dataframe 'user_vectorized_descriptions'
columns_components = ['user_desc_pca_%i' % i for i in range(len(pca.explained_variance_ratio_))]
df_components = pd.DataFrame(data = components, columns = columns_components)

In [ ]:
#Juntando os dataframes user_vectorized_descriptions com os valores das componentes do pca
user_vectorized_descriptions = pd.merge(user_vectorized_descriptions, df_components, left_index = True, right_index = True)

Juntando todos os dataframes

In [ ]:
#train_tweets com train_tweets_vectorized_media
train_tweets_medias = pd.merge(train_tweets, medias_without_duplicates,
                               left_on = "tweet_id", right_index = True)

In [ ]:
#train_tweets_medias com train_tweets_vectorized_text
train_tweets_medias_text = pd.merge(train_tweets_medias, train_tweets_vectorized_text,
                                    left_on='tweet_id', right_on='tweet_id', how = 'left')

In [ ]:
#train_tweets_medias_text com users
train_tmt_user = pd.merge(train_tweets_medias_text, users,
                          left_on='tweet_user_id', right_on='user_id')
train_tmt_user = train_tmt_user.drop(['user_id'], axis = 1)

In [ ]:
#train_tmt_user com user_vectorized_profile_images
train_tmt_user_images = pd.merge(train_tmt_user, user_vectorized_profile_images,
                                 left_on='tweet_user_id', right_on='user_id')
train_tmt_user_images = train_tmt_user_images.drop(['user_id'], axis = 1)

In [ ]:
#Merge com o user_vectorized_descriptions.csv
train_full = pd.merge(train_tmt_user_images, user_vectorized_descriptions,
                                 left_on='tweet_user_id', right_on='user_id')
train_full = train_full.drop(['user_id'], axis = 1)

In [ ]:
train_full.head(2)

In [ ]:
train_full.info()

Precisamos ainda, mudar algumas colunas do _train_tweets_. As colunas _tweet_hashtag_count_, _tweet_url_count_ e _tweet_mention_count_ são do tipo inteiro. Além disso, a coluna _tweet_has_attachment_ é do tipo booleana,  _tweet_attachment_class_ é uma variável dummy e a coluna _tweet_topic_ids_ é uma coluna de listas, com alguns valores nulos. Não vamos usar essa última coluna para treinar o modelo.  

In [ ]:
def fix_tweets(df):
    ''' Arruma as colunas tweet_hashtag_count, tweet_url_count, tweet_mention_count
        e tweet_attachment_class '''
    
    #Transformando o tipo das colunas
    cols = ['tweet_hashtag_count', 'tweet_url_count', 'tweet_mention_count']
    df[cols] = df[cols].applymap(np.int64)

    #Transformando a coluna 'tweet_attachment_class' em dummy
    dummy = pd.get_dummies(df['tweet_attachment_class'])
    df = pd.concat([df, dummy], axis = 1)
    df.drop('tweet_attachment_class', inplace=True, axis = 1)
    
    return df

train_full = fix_tweets(train_full)

### Ajustando o modelo

In [ ]:
from sklearn.model_selection import train_test_split,cross_val_score

#Selecionando as colunas
y = train_full.loc[:,'virality'].to_numpy()

#Variaveis que vamos usar para 'X'
colunas = [coluna for coluna in train_full.columns if coluna not in ['tweet_id', 'tweet_user_id', 'tweet_created_at_year',
                                                                     'tweet_created_at_month', 'tweet_created_at_day',
                                                                     'tweet_topic_ids', 'virality']]
X = train_full[colunas].to_numpy()

#Para calcularmos o score
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size= 0.8, random_state=7)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

rf_pipeline = Pipeline(steps = [('scale',StandardScaler()),('RF',RandomForestClassifier(random_state=7))])

#Ajustando o modelo
rf_pipeline.fit(X_train, y_train)
y_pred_rf = rf_pipeline.predict(X_test)
print('Acuracia Random Forest:',accuracy_score(y_pred_rf, y_test))

In [ ]:
from xgboost import XGBClassifier

xgboost_pipeline = Pipeline(steps = [('scale',StandardScaler()), ('XGB',XGBClassifier(random_state=7, eval_metric='mlogloss'))])

#Ajustando o modelo
xgboost_pipeline.fit(X_train, y_train)
y_pred_xgb = xgboost_pipeline.predict(X_test)
print('Acuracia XGBoost:',accuracy_score(y_pred_xgb, y_test))

Como o modelo de Random Forest obteve, no geral, uma acurácia maior, vamos fazer uma feature importance deste modelo.

In [ ]:
#Fazendo o feature importance
feature_importance=pd.DataFrame({'features':train_full[colunas].columns,
                                 'feature_importance':rf_pipeline.steps[1][1].feature_importances_})
feature_importance.sort_values('feature_importance',ascending=False)

O modelo possui 1558 features. Isso significa que, em média, cada feature contribui com 0.06% de importância. Há 277 features com importância acima da média. Essas features, sozinhas, possuem 66,64% da importância das features para o modelo.

In [ ]:
#Quao importante são as features acima da media
print(sum(feature_importance[feature_importance['feature_importance']>0.0006]['feature_importance']))

#Numero de features acima da media
feature_importance[feature_importance['feature_importance']>0.0006]

## Segunda solução


### Imports

In [ ]:
#Imports.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pandas_profiling import ProfileReport
from itertools import combinations_with_replacement
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import NuSVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
import warnings
from scipy.stats.stats import pearsonr  


### Carregando os dados

In [ ]:
#Tabelas de Treino.

#Tabela 1 Tweets.
dir_train_tweets = "../input/viral-tweets-prediction-dataset/Dataset/Tweets/train_tweets.csv"
data_train_tweets = pd.read_csv(dir_train_tweets,header = (0))
data_train_tweets.head(5)

In [ ]:
#Tabela 2 Users.
dir_users = "../input/viral-tweets-prediction-dataset/Dataset/Users/users.csv"
data_users = pd.read_csv(dir_users,header = (0))
data_users.head(5)

In [ ]:
#Tabela 3 Tweets Medias.
dir_tweets_medias = "../input/viral-tweets-prediction-dataset/Dataset/Tweets/train_tweets_vectorized_media.csv"
data_tweets_medias = pd.read_csv(dir_tweets_medias,header = (0))
data_tweets_medias.head(5)

<center>
    <hr >
    <h2> Tratamento das colunas </h2>
    <hr >
    <img src= 'https://miro.medium.com/max/2000/1*ke5xITbKv-QmjwYTjlpCgg.png' width="1000" >
    <hr >
</center>

Neste caso vamos usar apenas 3 das 6 tabelas, sendo elas:

    - Tweets
    - Users
    - tweet_vectorized_media
No entanto, vamos filtar algumas colunas de cada tabela e unir as tabelas gerando apenas 1 tabela para treinamento.

In [ ]:
#Tabela Tweets, vamos remover as colunas: 
columns_to_drop_tweets = [ "tweet_topic_ids",
                           "tweet_created_at_year",
                           "tweet_created_at_month",
                           "tweet_created_at_day",
                           "tweet_attachment_class",
                           "tweet_language_id",
                           "tweet_created_at_hour"]
#Remoção das colunas.
data_train_tweets = data_train_tweets.drop(columns = columns_to_drop_tweets)
#Renomeando a coluna tweet_user_id
data_train_tweets['user_id'] = data_train_tweets['tweet_user_id']
data_train_tweets = data_train_tweets.drop(columns = ['tweet_user_id'])
#Apresentação da tabela.
data_train_tweets.head(5)

In [ ]:
#Tabela User, vamos remover as colunas:
columns_to_drop_tweets = [ "user_created_at_year",
                           "user_created_at_month"]
#Remoção das colunas.
data_users = data_users.drop(columns = columns_to_drop_tweets)
#Apresentação da tabela.
data_users.head(5)

In [ ]:
#Tabela tweet_vectorized_media, vamos manter as colunas:
coluns_to_keep_tweet_vectorized_media = ["tweet_id",
                                         "media_id"]
#Selecionando apenas 2 colunas.
data_tweets_medias = data_tweets_medias[data_tweets_medias.columns[data_tweets_medias.columns.isin(coluns_to_keep_tweet_vectorized_media)]]
#Apresentação da tabela.
data_tweets_medias.head(5)

### Agora vamos criar a feature na Tabela Users:

    - user_followers/following_count = user_followers_count/user_following_count

In [ ]:
#Criação da feature.
data_users["user_followers/following_count"] = data_users["user_followers_count"]/data_users["user_following_count"]
#Apresentação da tabela.
data_users.head(5)

#### Criando a contagem de imagens

In [ ]:
#Lista de contagem de imagens.
n_imagens = data_tweets_medias["tweet_id"].value_counts()
tweet_id = list(n_imagens.to_frame().index)
count_images = list(n_imagens.to_frame().tweet_id)
#Adicionando ao data frame.
data_tweets_medias = pd.DataFrame({'tweet_id':tweet_id,'count_images':count_images})
#Apresentação da tabela.
data_tweets_medias.head(5)

#### Unindo as 3 tabelas

In [ ]:
#Unindo Tweets e Users.
data = pd.merge(data_train_tweets,data_users, on ='user_id', how = 'right')
#Unindo a anterior e Medias.
data = pd.merge(data,data_tweets_medias, on ='tweet_id', how = 'right')
#Apresentação da tabela.
data.head(5)

In [ ]:
data = data.replace(-np.inf, np.nan)
data = data.replace(np.inf, np.nan)

data = data.dropna()

data.head(5)

### Treinamento

In [ ]:
X = data.drop(columns = "virality")
y = data["virality"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
%%time

rfc = RandomForestClassifier()

rfc.fit(X_train, y_train)
y_pred= rfc.predict(X_test)

score_cross = (cross_val_score(rfc, X_train, y_train, cv=10))
print(round(np.mean(score_cross),2),"%")

print("Accuracy on Traing set: ",rfc.score(X_train,y_train))
print("Accuracy on Testing set: ",rfc.score(X_test,y_test))

In [ ]:
%%time

knn = KNeighborsClassifier()

knn.fit(X_train, y_train)
y_pred= knn.predict(X_test)

score_cross = (cross_val_score(knn, X_train, y_train, cv=10))
print(round(np.mean(score_cross),2),"%")

print("Accuracy on Traing set: ",knn.score(X_train,y_train))
print("Accuracy on Testing set: ",knn.score(X_test,y_test))

In [ ]:
%%time
warnings.filterwarnings('ignore')

xgb = XGBClassifier(eval_metric = 'mlogloss')

xgb.fit(X_train, y_train)
y_pred= xgb.predict(X_test)

score_cross = (cross_val_score(xgb, X_train, y_train, cv=10))
print(round(np.mean(score_cross),2),"%")

print("Accuracy on Traing set: ",xgb.score(X_train,y_train))
print("Accuracy on Testing set: ",xgb.score(X_test,y_test))

#### Utilizando o PCA


In [ ]:
X = data.drop(columns = "virality")
y = data["virality"]

#PCA
pca = PCA(n_components = 1)
pca.fit(X)

X = pca.transform(X)

print(np.round(pca.explained_variance_ratio_))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
rfc = KNeighborsClassifier()

rfc.fit(X_train, y_train)
y_pred= rfc.predict(X_test)

score_cross = (cross_val_score(rfc, X_train, y_train, cv=10))
print(round(np.mean(score_cross),2),"%")

print("Accuracy on Traing set: ",rfc.score(X_train,y_train))
print("Accuracy on Testing set: ",rfc.score(X_test,y_test))

In [ ]:
%%time
warnings.filterwarnings('ignore')

xgb = XGBClassifier(eval_metric = 'mlogloss')

xgb.fit(X_train, y_train.to_numpy().ravel())
y_pred= xgb.predict(X_test)

score_cross = (cross_val_score(xgb, X_train, y_train, cv=10))
print(round(np.mean(score_cross),2),"%")

print("Accuracy on Traing set: ",xgb.score(X_train,y_train))
print("Accuracy on Testing set: ",xgb.score(X_test,y_test))